```
███████  █████  ███████ ████████ ████████ ██████   █████  ███    ██ ███████ ███████ ███████ ██████  
██      ██   ██ ██         ██       ██    ██   ██ ██   ██ ████   ██ ██      ██      ██      ██   ██ 
█████   ███████ ███████    ██       ██    ██████  ███████ ██ ██  ██ ███████ █████   █████   ██████  
██      ██   ██      ██    ██       ██    ██   ██ ██   ██ ██  ██ ██      ██ ██      ██      ██   ██ 
██      ██   ██ ███████    ██       ██    ██   ██ ██   ██ ██   ████ ███████ ██      ███████ ██   ██ 

# Demo 1 - Source Oracle - Target PGSQL17

In [1]:
dotnet tool install --global Microsoft.dotnet-interactive --version 1.0.632301

Tool 'microsoft.dotnet-interactive' is already installed.


In [2]:
#r "nuget:Microsoft.DotNet.Interactive.PostgreSQL , *-*"

Installed Packages Microsoft.DotNet.Interactive.PostgreSQL, 1.0.0-beta.25323.1

Loading extension script from `C:\Users\romai\.nuget\packages\microsoft.dotnet.interactive.postgresql\1.0.0-beta.25323.1\interactive-extensions\dotnet\extension.dib`

Query PostgreSQL databases. 
 This extension adds support for connecting to PostgreSql databases using the #!connect postgres magic command.

## Working on TPCH SF10 ORDERS (15M rows)

### PGSQL 16 to PG 17

In [3]:
// Connect to PostgreSQL target database
#!connect postgres "Host=localhost;Port=25432;Database=tpch;Username=FastUser;Password=FastPassword" --kernel-name pgsql17

Kernel added: #!sql-pgsql17

#### Target orders_3 with a clustered ColumnStoreIndex

In [4]:
TRUNCATE TABLE tpch_10.orders;
SELECT count(*) count_target_table_rows from tpch_10.orders;

ALTER TABLE tpch_10.orders SET UNLOGGED;

count_target_table_rows
0


### Describe Oracle Source Table
```

In [5]:
echo 'DESC tpch_in.orders_flat;' | sqlplus -S TPCH_IN/TPCH_IN@//localhost:1521/ORCLPDB

 Nom					   NULL ?   Type
 ----------------------------------------- -------- ----------------------------
 O_ORDERKEY				   NOT NULL NUMBER(38)
 O_CUSTKEY				   NOT NULL NUMBER(19)
 O_ORDERSTATUS					    CHAR(1)
 O_TOTALPRICE					    NUMBER(19,3)
 O_ORDERDATE					    DATE
 O_ORDERPRIORITY				    CHAR(15)
 O_CLERK					    CHAR(15)
 O_SHIPPRIORITY 				    NUMBER(38)
 O_COMMENT					    VARCHAR2(79)



## Test with Method RowId (oracle source only)

In Oracle, a ROWID is a unique physical address identifying a row’s exact storage location.

You can SELECT it and even filter data using this hidden column. FastTransfer use the RowId column to select data by chunck on oracle source.

In [6]:

cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "oraodp" `
--sourceserver "localhost:1521/ORCLPDB" `
--sourcedatabase "ORCLPDB" `
--sourceuser "TPCH_IN" `
--sourcepassword "TPCH_IN" `
--sourceschema "TPCH_IN" `
--sourcetable "ORDERS_FLAT" `
`
--targetconnectiontype "pgcopy" `
--targetserver "localhost:25432" `
--targetuser "FastUser" `
--targetpassword "FastPassword" `
--targetdatabase "tpch" `
--targetschema "tpch_10" `
--targettable "orders" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Rowid" `
--degree -2 `
--runid "ORA21_to_PG17"

2025-09-24T12:00:30.094 +02:00 -|- FastTransfer -|- ORA21_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (25 day(s) left).
2025-09-24T12:00:30.103 +02:00 -|- FastTransfer -|- ORA21_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- Starting
2025-09-24T12:00:30.103 +02:00 -|- FastTransfer -|- ORA21_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-24T12:00:30.103 +02:00 -|- FastTransfer -|- ORA21_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- OS : Microsoft Windows 10.0.26100
2025-09-24T12:00:30.104 +02:00 -|- FastTransfer -|- ORA21_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- Process ID : 49052
2025-09-24T12:00:30.104 +02:00 -|- FastTransfer -|- ORA21_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- Provided Run ID : ORA21_to_PG17
2025-09-24T12:00:30.104 +02:00 -|- FastTransfer -|- ORA21_to_PG17 -|- INFORMATION -|

In [8]:
SELECT count(*) count_target_table_rows from tpch_10.orders;

count_target_table_rows
15000000


In [17]:
SELECT * FROM tpch_10.orders LIMIT 5;

o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
24552903,1313752,F,305754.99,1992-03-05 00:00:00Z,3-MEDIUM,Clerk#000007605,0,ly regular requests within the furiously regul
24575648,978844,F,175154.42,1992-12-10 00:00:00Z,2-HIGH,Clerk#000001182,0,y across the requests. quickly special accounts cajole fluffily
24575651,624299,F,116155.58,1992-02-09 00:00:00Z,4-NOT SPECIFIED,Clerk#000009852,0,ironic packages. fluffily express courts are across the slyly ironic theodo
24575682,201691,F,116158.29,1992-06-14 00:00:00Z,4-NOT SPECIFIED,Clerk#000004253,0,t the blithely final pinto beans; furio
24575712,1435832,F,330511.13,1992-11-27 00:00:00Z,1-URGENT,Clerk#000008569,0,its. regular foxes boost quickly. slyly special dugouts sleep blithe


#### Target orders_heap : Heap Table

In [10]:
DROP TABLE IF EXISTS tpch_10.orders_heap;

CREATE TABLE tpch_10.orders_heap AS
SELECT * 
FROM tpch_10.orders
WHERE 1=0;

ALTER TABLE tpch_10.orders_heap SET UNLOGGED;

SELECT count(*) count_orders_heap from tpch_10.orders_heap;

count_orders_heap
0


In [6]:

cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "oraodp" `
--sourceserver "localhost:1521/ORCLPDB" `
--sourcedatabase "ORCLPDB" `
--sourceuser "TPCH_IN" `
--sourcepassword "TPCH_IN" `
--sourceschema "TPCH_IN" `
--sourcetable "ORDERS_FLAT" `
`
--targetconnectiontype "pgcopy" `
--targetserver "localhost:25432" `
--targetuser "FastUser" `
--targetpassword "FastPassword" `
--targetdatabase "tpch" `
--targetschema "tpch_10" `
--targettable "orders_heap" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Rowid" `
--degree -2 `
--runid "ORA21_to_PG17_heap"

2025-09-22T23:57:35.147 +02:00 -|- FastTransfer -|- ORA21_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (27 day(s) left).
2025-09-22T23:57:35.155 +02:00 -|- FastTransfer -|- ORA21_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- Starting
2025-09-22T23:57:35.155 +02:00 -|- FastTransfer -|- ORA21_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-22T23:57:35.155 +02:00 -|- FastTransfer -|- ORA21_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- OS : Microsoft Windows 10.0.26100
2025-09-22T23:57:35.156 +02:00 -|- FastTransfer -|- ORA21_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- Process ID : 93976
2025-09-22T23:57:35.156 +02:00 -|- FastTransfer -|- ORA21_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- Provided Run ID : ORA21_to_PG17_heap
2025-09-22T23:57:35

In [12]:

SELECT count(*) count_rows_in_heap from tpch_10.orders_heap;

count_rows_in_heap
15000000


In [13]:
SELECT * from tpch_10.orders_heap LIMIT 5;

o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
31572036,630257,F,74496.62,1993-01-09 00:00:00Z,4-NOT SPECIFIED,Clerk#000007201,0,"slyly according to the regular, regular accounts. furiousl"
31572037,1226720,F,140196.16,1993-02-19 00:00:00Z,1-URGENT,Clerk#000004945,0,ar ideas use sometimes even requests. fluffily ruthless packages haggle.
9136614,1449592,F,157666.74,1993-12-30 00:00:00Z,4-NOT SPECIFIED,Clerk#000000520,0,he quickly final ideas cajole slyly across the regular realms. ent
31572069,1320178,F,70886.66,1993-05-20 00:00:00Z,4-NOT SPECIFIED,Clerk#000003731,0,arefully pending theodolit
31572070,215563,F,211005.1,1993-06-02 00:00:00Z,2-HIGH,Clerk#000004420,0,ronic deposits are. slyly si


#### Test using a bigger table : LineItem (60M Rows x 16 columns - 16GB) 

In [15]:
echo 'DESC tpch_in.lineitem_iot' | sqlplus -S TPCH_IN/TPCH_IN@//localhost:1521/ORCLPDB

 Nom					   NULL ?   Type
 ----------------------------------------- -------- ----------------------------
 L_ORDERKEY				   NOT NULL NUMBER
 L_PARTKEY				   NOT NULL NUMBER
 L_SUPPKEY				   NOT NULL NUMBER
 L_LINENUMBER				   NOT NULL NUMBER
 L_QUANTITY				   NOT NULL NUMBER(12,2)
 L_EXTENDEDPRICE			   NOT NULL NUMBER(15,2)
 L_DISCOUNT				   NOT NULL NUMBER(4,2)
 L_TAX					   NOT NULL NUMBER(4,2)
 L_RETURNFLAG				   NOT NULL CHAR(1)
 L_LINESTATUS				   NOT NULL CHAR(1)
 L_SHIPDATE				   NOT NULL DATE
 L_COMMITDATE				   NOT NULL DATE
 L_RECEIPTDATE				   NOT NULL DATE
 L_SHIPINSTRUCT 			   NOT NULL CHAR(25)
 L_SHIPMODE				   NOT NULL CHAR(10)
 L_COMMENT					    VARCHAR2(44)



In [20]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "oraodp" `
--sourceserver "localhost:1521/ORCLPDB" `
--sourcedatabase "ORCLPDB" `
--sourceuser "TPCH_IN" `
--sourcepassword "TPCH_IN" `
--sourceschema "TPCH_IN" `
--sourcetable "LINEITEM" `
`
--targetconnectiontype "pgcopy" `
--targetserver "localhost:25432" `
--targetuser "FastUser" `
--targetpassword "FastPassword" `
--targetdatabase "tpch" `
--targetschema "tpch_10" `
--targettable "lineitem_heap_unlogged" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Rowid" `
--degree -2 `
--runid "ORA21_to_PG17_60M_rowsx16_columns"

2025-09-23T00:06:53.286 +02:00 -|- FastTransfer -|- ORA21_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (27 day(s) left).
2025-09-23T00:06:53.296 +02:00 -|- FastTransfer -|- ORA21_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- Starting
2025-09-23T00:06:53.296 +02:00 -|- FastTransfer -|- ORA21_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-23T00:06:53.296 +02:00 -|- FastTransfer -|- ORA21_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- OS : Microsoft Windows 10.0.26100
2025-09-23T00:06:53.297 +02:00 -|- FastTransfer -|- ORA21_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- Process ID : 79164
2025-09-23T00:06:53.297 +02:00 -|- FastTransfe

#### Results with Oracle Sources:

| ORA21c ==> PG17  				| Elapsed Time (seconds)|Cells/sec 	    |Parallel Dispatch Method | degree |
|-----------------------------------------------------------------------|:---------------------:|----------:| :--: | :--: |
| FastTransfer Load LINEITEM (60M Rows x 16c columns) HEAP Target       | 65.6s     			|15_212_496    | Rowid | 16 (-2) |
| FastTransfer Load ORDERS   (15M Rows x  9c columns) HEAP Target 		| 10.9s     			|14_331_311     | Rowid | 16 (-2) | 
